In [79]:
#Dependencies
import gmaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy.stats import linregress
from math import *
import datetime

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
# The path to our bus stop locations CSV file
locations_file = "Resources/stops.csv"

# Read our file data into pandas
locations_df = pd.read_csv(locations_file)
locations_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN
1,2.0,NaN,10800 BROOKPARK RD (HOME DEPOT),NaN,41.418571,-81.760947,NaN,NaN,0,NaN
2,4.0,NaN,EUCLID AV & E 100TH ST,NaN,41.503697,-81.618108,NaN,NaN,0,NaN
3,5.0,NaN,10081 W RIDGEWOOD DR,NaN,41.384506,-81.756317,NaN,NaN,0,NaN
4,14.0,NaN,PROSPECT AV & ONTARIO ST,NaN,41.498030,-81.692012,NaN,NaN,0,NaN


In [3]:
# The path to our bus stop times CSV file
stops_file = "Resources/stop_times.csv"

# Read our file data into pandas
stops_df = pd.read_csv(stops_file)
stops_df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,16741667,3:21:00 AM,3:21:00 AM,14507,1,0,0
1,16741667,3:22:00 AM,3:22:00 AM,4568,2,0,0
2,16741667,3:23:00 AM,3:23:00 AM,4522,3,0,0
3,16741667,3:24:00 AM,3:24:00 AM,4378,4,0,0
4,16741667,3:25:00 AM,3:25:00 AM,4556,5,0,0


In [4]:
# merge
bus_data = pd.merge(locations_df, stops_df, on=["stop_id", "stop_id"])
bus_data.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,trip_id,arrival_time,departure_time,stop_sequence,pickup_type,drop_off_type
0,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741990,5:54:00 AM,5:54:00 AM,82,0,0
1,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741993,6:54:00 AM,6:54:00 AM,82,0,0
2,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741995,7:25:00 AM,7:25:00 AM,82,0,0
3,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741997,7:57:00 AM,7:57:00 AM,82,0,0
4,1.0,NaN,COLUMBUS ST & BROADWAY,NaN,41.392985,-81.536518,NaN,NaN,0,NaN,16741999,8:28:00 AM,8:28:00 AM,82,0,0


In [5]:
#groupby stop id to see how many stops 
grouped_bystops = bus_data.groupby(['stop_id', 'stop_lat', 'stop_lon'])
numberstops = grouped_bystops['arrival_time'].nunique()
numberstops

stop_id  stop_lat   stop_lon  
1.0      41.392985  -81.536518     63
2.0      41.418571  -81.760947     72
4.0      41.503697  -81.618108    241
5.0      41.384506  -81.756317    108
14.0     41.498030  -81.692012    370
                                 ... 
50001.0  41.500290  -81.691917    492
60001.0  41.500764  -81.682781    172
60002.0  41.500256  -81.686436    166
60003.0  41.499918  -81.688906    166
60004.0  41.499515  -81.691994    161
Name: arrival_time, Length: 5799, dtype: int64

In [6]:
#numberstops.tolist()

In [7]:
stopnum_df = pd.DataFrame({"Number of Stops":numberstops})
stopnum_df

,,,Number of Stops
stop_id,stop_lat,stop_lon,
1.0,41.392985,-81.536518,63
2.0,41.418571,-81.760947,72
4.0,41.503697,-81.618108,241
5.0,41.384506,-81.756317,108
14.0,41.498030,-81.692012,370
...,...,...,...
50001.0,41.500290,-81.691917,492
60001.0,41.500764,-81.682781,172
60002.0,41.500256,-81.686436,166


In [8]:
# merge stops with lat and long
stopnum_df.reset_index(inplace = True)


In [9]:
# Store lat and long
locations = stopnum_df[['stop_lat','stop_lon']]
stops = stopnum_df[['Number of Stops']].astype(float)

In [10]:
# Plot Heatmap
fig_heat = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=2, point_radius=.0025)
fig_heat.add_layer(heat_layer)
fig_heat

Figure(layout=FigureLayout(height='420px'))

In [11]:
# plot locations
fig_mark = gmaps.figure()
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig_mark.add_layer(markers)
fig_mark

Figure(layout=FigureLayout(height='420px'))

In [92]:
# Add destination coordinates for Public Square
dest_coords = (41.4997,81.6937)

#stopnum_df["Distance (m)"] = ""

In [93]:
#Drop rows with bad data
clean_trip = stops_df.copy()
clean_trip.dropna(how="any",inplace=True)
clean_trip

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,16741667,3:21:00 AM,3:21:00 AM,14507,1,0,0
1,16741667,3:22:00 AM,3:22:00 AM,4568,2,0,0
2,16741667,3:23:00 AM,3:23:00 AM,4522,3,0,0
3,16741667,3:24:00 AM,3:24:00 AM,4378,4,0,0
4,16741667,3:25:00 AM,3:25:00 AM,4556,5,0,0
...,...,...,...,...,...,...,...
953813,16803524,11:55:00 PM,11:55:00 PM,21023,80,0,0
953814,16803524,11:56:00 PM,11:56:00 PM,14159,81,0,0
953815,16803524,11:56:00 PM,11:56:00 PM,14602,82,0,0
953816,16803524,11:57:00 PM,11:57:00 PM,14603,83,0,0


In [94]:
clean_trip.set_index("arrival_time")
clean_trip.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,16741667,3:21:00 AM,3:21:00 AM,14507,1,0,0
1,16741667,3:22:00 AM,3:22:00 AM,4568,2,0,0
2,16741667,3:23:00 AM,3:23:00 AM,4522,3,0,0
3,16741667,3:24:00 AM,3:24:00 AM,4378,4,0,0
4,16741667,3:25:00 AM,3:25:00 AM,4556,5,0,0


In [95]:
#Convert arrival and departure times into timestamps
clean_trip["arrival_time"] = pd.to_datetime(clean_trip["arrival_time"])
clean_trip["departure_time"] = pd.to_datetime(clean_trip["departure_time"])
clean_trip

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,16741667,2020-09-10 03:21:00,2020-09-10 03:21:00,14507,1,0,0
1,16741667,2020-09-10 03:22:00,2020-09-10 03:22:00,4568,2,0,0
2,16741667,2020-09-10 03:23:00,2020-09-10 03:23:00,4522,3,0,0
3,16741667,2020-09-10 03:24:00,2020-09-10 03:24:00,4378,4,0,0
4,16741667,2020-09-10 03:25:00,2020-09-10 03:25:00,4556,5,0,0
...,...,...,...,...,...,...,...
953813,16803524,2020-09-10 23:55:00,2020-09-10 23:55:00,21023,80,0,0
953814,16803524,2020-09-10 23:56:00,2020-09-10 23:56:00,14159,81,0,0
953815,16803524,2020-09-10 23:56:00,2020-09-10 23:56:00,14602,82,0,0
953816,16803524,2020-09-10 23:57:00,2020-09-10 23:57:00,14603,83,0,0


In [131]:
trip_group = clean_trip.groupby("trip_id").count()
trip_group["min_arr"] = ""
trip_group["max_arr"] = ""
trip_group.head()

,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,min_arr,max_arr
trip_id,,,,,,,,
16723936,18,18,18,18,18,18,,
16723937,18,18,18,18,18,18,,
16723938,18,18,18,18,18,18,,
16723939,18,18,18,18,18,18,,
16723940,23,23,23,23,23,23,,


In [129]:
#Make sure that trips are one way
clean_trip.loc[clean_trip["trip_id"] == 16723936]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
504408,16723936,2020-09-10 09:30:00,2020-09-10 09:30:00,18915,1,0,0
504409,16723936,2020-09-10 09:33:00,2020-09-10 09:33:00,28886,2,0,0
504410,16723936,2020-09-10 09:36:00,2020-09-10 09:36:00,30879,3,0,0
504411,16723936,2020-09-10 09:39:00,2020-09-10 09:39:00,28923,4,0,0
504412,16723936,2020-09-10 09:41:00,2020-09-10 09:41:00,10960,5,0,0
504413,16723936,2020-09-10 09:43:00,2020-09-10 09:43:00,10962,6,0,0
504414,16723936,2020-09-10 09:45:00,2020-09-10 09:45:00,10963,7,0,0
504415,16723936,2020-09-10 09:46:00,2020-09-10 09:46:00,10954,8,0,0
504416,16723936,2020-09-10 09:48:00,2020-09-10 09:48:00,10937,9,0,0
504417,16723936,2020-09-10 09:49:00,2020-09-10 09:49:00,10936,10,0,0


In [132]:
trip_group.reset_index(inplace=True)
#trip_group.head()

min_trips = clean_trip.groupby(["trip_id"]).agg({"arrival_time":["min","max"]})
min_trips

arrival_time                    
                         min                 max
trip_id                                         
16723936 2020-09-10 09:30:00 2020-09-10 09:59:00
16723937 2020-09-10 09:52:00 2020-09-10 10:24:00
16723938 2020-09-10 09:22:00 2020-09-10 09:54:00
16723939 2020-09-10 21:22:00 2020-09-10 21:54:00
16723940 2020-09-10 17:05:00 2020-09-10 17:45:00
...                      ...                 ...
16811833 2020-09-10 17:40:00 2020-09-10 18:52:00
16811837 2020-09-10 16:35:00 2020-09-10 17:16:00
16811838 2020-09-10 18:05:00 2020-09-10 18:46:00
16811842 2020-09-10 16:40:00 2020-09-10 17:25:00
16811843 2020-09-10 18:08:00 2020-09-10 18:50:00

[16057 rows x 2 columns]

In [139]:
min_trips["duration"] = min_trips["arrival_time"]["max"] - min_trips["arrival_time"]["min"]

min_trips

arrival_time                     duration
                         min                 max         
trip_id                                                  
16723936 2020-09-10 09:30:00 2020-09-10 09:59:00 00:29:00
16723937 2020-09-10 09:52:00 2020-09-10 10:24:00 00:32:00
16723938 2020-09-10 09:22:00 2020-09-10 09:54:00 00:32:00
16723939 2020-09-10 21:22:00 2020-09-10 21:54:00 00:32:00
16723940 2020-09-10 17:05:00 2020-09-10 17:45:00 00:40:00
...                      ...                 ...      ...
16811833 2020-09-10 17:40:00 2020-09-10 18:52:00 01:12:00
16811837 2020-09-10 16:35:00 2020-09-10 17:16:00 00:41:00
16811838 2020-09-10 18:05:00 2020-09-10 18:46:00 00:41:00
16811842 2020-09-10 16:40:00 2020-09-10 17:25:00 00:45:00
16811843 2020-09-10 18:08:00 2020-09-10 18:50:00 00:42:00

[16057 rows x 3 columns]